# KeyNote Chunking

In [1]:
import os
import nltk
import pandas as pd

# NLTK 문장 분리 데이터 다운로드 (최초 한 번만 필요)
try:
    nltk.data.find('tokenizers/punkt')
except nltk.downloader.DownloadError:
    print("NLTK 'punkt' 데이터를 다운로드합니다...")
    nltk.download('punkt', quiet=True)
    print("다운로드 완료!")

RAW_TEXTS_DIR = "raw_texts"

def process_keynotes_into_sentences(directory):
    """
    폴더 안의 모든 txt 파일을 읽어, 문장 단위로 분리하고 DataFrame으로 만듭니다.
    """
    if not os.path.exists(directory):
        print(f"[오류] '{directory}' 폴더를 찾을 수 없습니다.")
        print("Colab 왼편 파일 탐색기에서 폴더를 만들고, 6개의 키노트 txt 파일을 업로드해주세요.")
        return None

    all_sentences_data = []
    
    print(f"'{directory}' 폴더에서 파일들을 처리합니다...")
    # 폴더 안의 모든 파일을 순회
    for filename in sorted(os.listdir(directory)):
        if filename.endswith(".txt"):
            filepath = os.path.join(directory, filename)
            
            with open(filepath, 'r', encoding='utf-8') as f:
                # 파일 전체 내용을 하나의 텍스트로 읽음
                full_text = f.read()
            
            # NLTK를 사용해 텍스트를 문장 리스트로 분할
            sentences = nltk.sent_tokenize(full_text)
            
            # 각 문장을 데이터로 저장
            for i, sentence in enumerate(sentences):
                cleaned_sentence = sentence.replace('\n', ' ').strip()
                if cleaned_sentence:
                    all_sentences_data.append({
                        "source_file": filename,
                        "sentence_index": i + 1,
                        "sentence_text": cleaned_sentence,
                        "word_count": len(cleaned_sentence.split())
                    })
    
    if not all_sentences_data:
        print("[결과] 처리할 텍스트 파일이 없습니다. 폴더에 .txt 파일을 추가해주세요.")
        return None
        
    # 리스트를 pandas DataFrame으로 변환
    df = pd.DataFrame(all_sentences_data)
    return df

# --- 메인 실행 ---
df_sentences = process_keynotes_into_sentences(RAW_TEXTS_DIR)

# 결과 출력
if df_sentences is not None:
    print(f"\n✅ 총 {len(df_sentences)}개의 문장을 성공적으로 분리했습니다.")
    print("--- 분리된 문장 데이터 (앞 20개) ---")
    # DataFrame을 표 형태로 예쁘게 출력
    display(df_sentences.head(20))

'raw_texts' 폴더에서 파일들을 처리합니다...

✅ 총 6039개의 문장을 성공적으로 분리했습니다.
--- 분리된 문장 데이터 (앞 20개) ---


,source_file,sentence_index,sentence_text,word_count
0,Apple Music Event 2001.txt,1,Thanks for coming today.,4
1,Apple Music Event 2001.txt,2,"So, we have something really exciting for you ...",9
2,Apple Music Event 2001.txt,3,Now Apple's been doing a lot of great stuff la...,10
3,Apple Music Event 2001.txt,4,"We shipped Mac 10.1 last month, an incredibly ...",29
4,Apple Music Event 2001.txt,5,"We updated our whole portable line last week, ...",20
5,Apple Music Event 2001.txt,6,And we lured you here today with the promise o...,19
6,Apple Music Event 2001.txt,7,And that's exactly what we're going to do.,8
7,Apple Music Event 2001.txt,8,"So, let's start with the digital hub.",7
8,Apple Music Event 2001.txt,9,This is a strategy that we announced in Januar...,16
9,Apple Music Event 2001.txt,10,And that strategy is that we believe the Mac c...,17


In [ ]:
RAW_TEXTS_DIR = "finetuning/raw_texts"

def reformat_files_by_sentence(directory):
    """
    폴더 안의 모든 txt 파일을 읽어, '한 문장당 한 줄' 형식으로 덮어씁니다.
    """
    if not os.path.exists(directory):
        print(f"[오류] '{directory}' 폴더를 찾을 수 없습니다.")
        print("Colab 왼편 파일 탐색기에서 폴더를 만들고, 6개의 키노트 txt 파일을 업로드해주세요.")
        return

    print(f"'{directory}' 폴더 안의 파일들을 변환합니다...")
    
    file_count = 0
    # 폴더 안의 모든 파일을 순회
    for filename in sorted(os.listdir(directory)):
        if filename.endswith(".txt"):
            file_count += 1
            filepath = os.path.join(directory, filename)
            
            print(f"  - [{filename}] 파일 처리 중...")
            
            # 1. 원본 파일 내용 전체를 읽어옴
            with open(filepath, 'r', encoding='utf-8') as f:
                full_text = f.read()
            
            # 2. NLTK를 사용해 텍스트를 문장 리스트로 분할
            sentences = nltk.sent_tokenize(full_text)
            
            # 3. 분리된 문장들을 개행(\n)으로 합쳐서 새로운 내용을 만듬
            #    (중간에 포함된 불필요한 개행 문자는 공백으로 치환)
            reformatted_text = "\n".join([s.replace('\n', ' ').strip() for s in sentences if s.strip()])
            
            # 4. 원본 파일을 쓰기('w') 모드로 열어, 새로운 내용으로 덮어씀
            with open(filepath, 'w', encoding='utf-8') as f:
                f.write(reformatted_text)
            
            print(f"    -> {len(sentences)}개 문장으로 분리하여 덮어쓰기 완료.")

    if file_count > 0:
        print(f"\n✅ 총 {file_count}개의 파일 변환을 완료했습니다.")
    else:
        print("\n[결과] 처리할 텍스트 파일이 없습니다. 폴더에 .txt 파일을 추가해주세요.")


# --- 메인 실행 ---
reformat_files_by_sentence(RAW_TEXTS_DIR)

'raw_texts' 폴더 안의 파일들을 변환합니다...
  - [Apple Music Event 2001.txt] 파일 처리 중...
    -> 703개 문장으로 분리하여 덮어쓰기 완료.
  - [Apple Music Event 2005.txt] 파일 처리 중...
    -> 708개 문장으로 분리하여 덮어쓰기 완료.
  - [Macworld 2007.txt] 파일 처리 중...
    -> 1715개 문장으로 분리하여 덮어쓰기 완료.
  - [Macworld 2008.txt] 파일 처리 중...
    -> 1018개 문장으로 분리하여 덮어쓰기 완료.
  - [Steve Jobs' 2005 Stanford Commencement Address.txt] 파일 처리 중...
    -> 142개 문장으로 분리하여 덮어쓰기 완료.
  - [WWDC 2010.txt] 파일 처리 중...
    -> 1394개 문장으로 분리하여 덮어쓰기 완료.
  - [iMac Introduction Event 1998.txt] 파일 처리 중...
    -> 359개 문장으로 분리하여 덮어쓰기 완료.

✅ 총 7개의 파일 변환을 완료했습니다.
